In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
from collections import defaultdict 
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv("temp_data3.csv")
df=df.iloc[:4000,:]

In [ ]:
list1=[]
list1=df.Sentiment.values
for i in range(len(list1)):
    if(list1[i]=="mixed"):
        df.drop([i],axis=0,inplace=True)
    elif(list1[i]=="neutral"):
        df.drop([i],axis=0,inplace=True)

df["Sentiment"].value_counts()

In [ ]:
text=df["cleaned_text"]
sentiment=df["Sentiment"]
dataf=pd.concat([text,sentiment],axis=1)

In [ ]:
data=dataf["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
data=data.str.replace("[^\w\s]","")#harf dışı simgeler,noktalama
data= data.str.replace(" \d+", " ")#sayılar 
from nltk.corpus import stopwords
nltk.download("stopwords")
duraklama_kelimeleri = stopwords.words()
data=pd.DataFrame(data,columns=["cleaned_text"])
data=data["cleaned_text"].apply(lambda x: " ".join(x for x in x.split() if x not in duraklama_kelimeleri))
data=pd.concat([data,sentiment],axis=1)

In [ ]:
cleaned_text = data['cleaned_text'].values
labels = data['Sentiment'].values

In [ ]:
cleaned_text=np.array(cleaned_text)

In [ ]:
nltk.download('punkt')
#clean the corpus.
sentences = []
vocab = []
for sent in cleaned_text:
    x = word_tokenize(sent)
    sentence = [w.lower() for w in x if w.isalpha() ]# harf olup ulmadığını kontrol eder isalpha
    sentences.append(sentence)
    for word in sentence:
        if word not in vocab:
            vocab.append(word)
len_vector = len(vocab)

In [ ]:
index_word = {}
i = 0
for word in vocab:
    index_word[word] = i 
    i += 1

In [ ]:
sentences[1]

In [ ]:
count_dict = defaultdict(int)
vec = np.zeros(len_vector)
for item in sentences[1]:
    count_dict[item] += 1
for key,item in count_dict.items():
    vec[index_word[key]] = item

In [ ]:
index_word

In [ ]:
count_dict

In [ ]:
def bag_of_words(sent):
    count_dict = defaultdict(int)
    vec = np.zeros(len_vector)
    for item in sent:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_word[key]] = item
    return vec   

In [ ]:
vocab

In [ ]:
tablo = np.zeros((len(sentences),len(vocab)))
tablo =pd.DataFrame(tablo,columns=vocab)
i=0
j=0
for i in range(len(vocab)):
    if i<len(sentences):
        vector = bag_of_words(sentences[j])
        tablo.loc[i] = vector
        j+=1
vector = vector.astype("float64")
tablo

In [ ]:
len(vector)

In [ ]:
tablo = tablo.astype("float64")

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(tablo, encoded_labels, stratify = labels)

In [ ]:
def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output

In [ ]:
init_parameters = {} 
init_parameters["weight"] = np.zeros(train_sentences.shape[1])
init_parameters["bias"] = 0

In [ ]:
len(train_sentences)

In [ ]:
def optimize(x, y,learning_rate,iterations,parameters): 
    size = x.shape[0]#train sayısı
    weight = parameters["weight"] 
    bias = parameters["bias"]
    loss_tr=[]
    accuracy_tr=[]
    iterations2=[]
    for i in range(iterations): 
        sigma = sigmoid(np.dot(x, weight) + bias)
        loss = -1/size * np.sum(y * np.log(sigma)) + (1 - y) * np.log(1-sigma)
        loss_tr.append(max(loss))
        dW = 1/size * np.dot(x.T, (sigma - y))
        db = 1/size * np.sum(sigma - y)
        weight1=weight
        weight -= learning_rate * dW
        delta_weight=weight-weight1
        bias1=bias
        bias -= learning_rate * db 
        delta_bias=bias-bias1
        
        print(delta_weight)
    
        parameters["weight"] = weight
        parameters["bias"] = bias
        output_values_tr = np.dot(train_sentences, parameters["weight"]) + parameters["bias"]
        predictions_tr = sigmoid(output_values_tr) >= 0.54
        accuracy_tr.append(accuracy_score(train_labels,predictions_tr))
        iterations2.append(i)  
    return parameters,loss_tr,accuracy_tr,accuracy_te,iterations2

In [ ]:
def test(x, y,parameters,learning_rate): 
    size = x.shape[0]#train sayısı
    weight = parameters["weight"] 
    bias = parameters["bias"]
    loss_te=[]
    accuracy_te=[]
    sentences=[]
    for i in range(len(x)): 
        sigma = sigmoid(np.dot(x, weight) + bias)
        loss = -1/size * np.sum(y * np.log(sigma)) + (1 - y) * np.log(1-sigma)
        loss_te.append(max(loss))
        dW = 1/size * np.dot(x.T, (sigma - y))
        db = 1/size * np.sum(sigma - y)
        weight -= learning_rate * dW
        bias -= learning_rate * db 
    
        parameters["weight"] = weight
        parameters["bias"] = bias
        
        output_values_te = np.dot(test_sentences, parameters["weight"]) + parameters["bias"]
        predictions_te = sigmoid(output_values_te) >= 0.54
        accuracy_te.append(accuracy_score(test_labels,predictions_te))
        sentences.append(i)
        
    return loss_te,accuracy_te,predictions_te,sentences

In [ ]:
def train(x, y, learning_rate,iterations):
    parameters_out,loss2, accuracy_tr,accuracy_te,iterations2 = optimize(x, y, learning_rate, iterations ,init_parameters)
    return parameters_out,loss2,accuracy_tr,accuracy_te,iterations2

In [ ]:
parameters_out,loss_tr,accuracy_tr,accuracy_te,iterations2 = train(train_sentences, train_labels, 
                                                                 learning_rate = 0.02, iterations = 100)

In [ ]:
loss_te,accuracy_te,predictions_te,sentences_te=test(test_sentences,test_labels,parameters_out,learning_rate = 0.02)

In [ ]:
plt.scatter(iterations2,accuracy_tr)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'accuracy_tr' )  
plt.title("0.02-400-Bow-Acc_Tr")
plt.savefig("0.02-400-Bow-Acc_Tr.png")
plt.show()

In [ ]:
plt.scatter(iterations2,loss_tr)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'loss' )  
plt.title("0.02-400-Bow-Loss_Tr")
plt.savefig("0.02-400-Bow-Loss_Tr.png")
plt.show()  

In [ ]:
plt.scatter(sentences_te,loss_te) 
plt.xlabel( 'test_sentences' ) 
plt.ylabel( 'loss_te' )  
plt.title("0.02-400-Bow-Loss_Te")
plt.savefig("0.02-400-Bow-Loss_Te.png")
plt.show() 

In [ ]:
plt.scatter(sentences_te,accuracy_te)  
plt.xlabel( 'sentences_te' ) 
plt.ylabel( 'accuracy' )  
plt.title("0.02-400-Bow-Acc_Te")
plt.savefig("0.02-400-Bow-Acc_Te.png")
plt.show()  

#### Lr=0.05, iterations=400

In [ ]:
output_values = np.dot(test_sentences, parameters_out["weight"]) + parameters_out["bias"]
a=sigmoid(output_values)
a.mean()
predictions = sigmoid(output_values) >= 0.54
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,predictions))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

class_names=["0","1"]

print(classification_report(test_labels, predictions, target_names=class_names))

print(confusion_matrix(test_labels, predictions))

In [ ]:
pred=[]
for i in range(len(predictions)):
    if(predictions[i]==True):
        pred.append("1")
    else: pred.append(0)
x= zip(pred,test_labels)
for i,j in x:
    print("Tahmin: {} Gerçek:{}".format(i,j))